# Package loading

In [86]:
# Install the titlecase library
!pip install titlecase


  Using cached titlecase-2.4.1.tar.gz (14 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for titlecase: filename=titlecase-2.4.1-py2.py3-none-any.whl size=10991 sha256=2e88e557e078cf2fb254a7f5a3028e40abdea343384ed33b6965573096eec3dc
  Stored in directory: c:\users\d e l l\appdata\local\pip\cache\wheels\10\7b\73\48cd4db360964e4f7e8d7cdb87f6b84254efac62e75ff07519
Successfully built titlecase


In [87]:
!pip install faker tqdm

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.8 MB 2.4 MB/s eta 0:00:01
   ---------------------------- ----------- 1.3/1.8 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 2.8 MB/s eta 0:00:00


In [88]:
pip install fastcluster

Note: you may need to restart the kernel to use updated packages.


In [89]:
pip install langdetect fuzzywuzzy[speedup] python-Levenshtein


     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     -------------------------------------- 981.5/981.5 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------------------- -------------- 1.0/1.6 MB 7.2 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 5.8 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993251 sha256=aa02c2285ceffdf71e13a17f6151b2c8b48839932e1963f442e428199a67fd29
  Stored in directory: c:\users\d e l l\appdata\local\pip\cache\wheels\d1\c1\d9\7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [90]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [91]:
!pip install scikit-surprise

In [92]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [93]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import os
from scipy.sparse import coo_matrix

# Data loading

In [96]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
dataset_data= pd.read_csv('/content/drive/MyDrive/book-rec-collab/NEW.csv')

In [ ]:
dataset_data.head(2)

In [ ]:
dataset_data.isnull().sum()

In [ ]:
dataset_data.duplicated().sum()

In [ ]:
dataset_data.drop_duplicates(inplace=True)

In [ ]:
dataset_data.columns

In [ ]:
column_name_mapping = {

    'User':'userID',
    'profileName' : 'userName',
    'ISBN' : 'bookID',
    'authors_split': 'authors',
    'categories_split':'category_list'
}

dataset_data.rename(columns=column_name_mapping, inplace=True)

In [ ]:
dataset_data.columns

# Pre processing

In [ ]:
dataset_data['bookTitle'] = dataset_data['Title']
dataset_data['bookAuthor'] = dataset_data['authors']
dataset_data['bookCategory'] = dataset_data['category_list']
dataset_data['bookPublisher']=dataset_data['publisher']

In [ ]:
dataset_data['bookTitle']=dataset_data['bookTitle'].astype(str)
dataset_data['bookAuthor']=dataset_data['bookAuthor'].astype(str)
dataset_data['bookCategory']=dataset_data['bookCategory'].astype(str)
dataset_data['bookPublisher']=dataset_data['bookPublisher'].astype(str)

In [ ]:
from titlecase import titlecase

# Apply the titlecase function to the 'title' column
dataset_data['bookTitle'] = dataset_data['bookTitle'].apply(titlecase)


In [ ]:
dataset_data['bookAuthor'] = dataset_data['bookAuthor'].apply(titlecase)
dataset_data['bookCategory'] = dataset_data['bookCategory'].apply(titlecase)
dataset_data['bookPublisher'] = dataset_data['bookPublisher'].apply(titlecase)

Title cleaning

In [ ]:
# remove special characters from book title
import string
dataset_data['Title'] = dataset_data['Title'].str.translate(str.maketrans('', '', string.punctuation))


In [ ]:
#rim whitespace
dataset_data['Title'] = dataset_data['Title'].str.strip()


In [ ]:
#remove common words like "a", "an"
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
dataset_data['Title'] = dataset_data['Title'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [ ]:
# lemmatization/ stemming to reduce words to thir base form
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
dataset_data['dataset_data'] = dataset_data['Title'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))


Identify duplicate books

In [ ]:
# Identify and handle empty processed titles
empty_titles = dataset_data[dataset_data['Title'] == '']
print("Number of titles reduced to empty strings:", len(empty_titles))
# Decide to remove or retain these entries
dataset_data = dataset_data[dataset_data['Title'] != '']

checking about empty string titled books

In [ ]:
records = pd.read_csv('/content/records.csv')

In [ ]:
empty_title_rows = records[dataset_data['Title'] == '']

In [ ]:
num_empty_titles = len(empty_title_rows)
total_titles = len(records)
print(f"Number of titles reduced to empty strings: {num_empty_titles} out of {total_titles}")

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


In [ ]:
def find_matches(row, choices, threshold=80):
    match = process.extractOne(row['Title'], choices, scorer=fuzz.token_sort_ratio)
    if match[1] >= threshold:
        return match[0]
    else:
        return row['Title']
unique_titles = records['Title'].unique()
records['canonical_title'] = records.apply(find_matches, axis=1, choices=unique_titles)


In [ ]:
records.columns

In [ ]:
records['canonical_title'].nunique()

In [ ]:
#cluster titles
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(records['canonical_title'])

k = 50  #number of clusters
model = KMeans(n_clusters=k)
model.fit(X)

records['cluster'] = model.labels_


In [ ]:
#create canonical_book_id
records['canonical_book_id'] = records.groupby('canonical_title').ngroup()

In [ ]:
copied = records.copy()

In [ ]:
dataset_data=copied.copy()

In [ ]:
book_meta = records.groupby('canonical_book_id').agg({
    'userID': 'first',
    'userName': 'first',
    'rating': 'first',
    'bookTitle': 'first',
    'bookAuthor': 'first',
    'bookCategory': 'first',
    'bookPublisher': 'first',
    'description': 'first',
    'image': 'first',
    'rating': 'first',
    'Title': 'first',
    'authors': 'first',
    'publishedDate': 'first',
    'Price': 'first',
    'publisher': 'first',
    'categories': 'first',
    # Add other fields as necessary
}).reset_index()


In [ ]:
#chceck duplicates for same user rating for same book with canonical_book_id
duplicates = records[records.duplicated(subset=['userID','userName', 'canonical_book_id','authors','Price','publisher'], keep=False)]


In [ ]:
#resolve duplicates
records = records.groupby(['userID', 'canonical_book_id']).agg({
    'userName': 'first',
    'bookID': 'first',
    'rating': 'first',
    'bookTitle': 'first',
    'bookAuthor': 'first',
    'bookCategory': 'first',
    'bookPublisher': 'first',
    'description': 'first',
    'image': 'first',
    'rating': 'first',
    'Title': 'first',
    'authors': 'first',
    'publishedDate': 'first',
    'Price': 'first',
    'publisher': 'first',
    'categories': 'first',
}).reset_index()

In [ ]:
records.to_csv('data.csv', index=False)

Final cleaning

In [ ]:
# Number of unique bookIDs
num_bookIDs = records['bookID'].nunique()

# Number of unique canonical_book_ids
num_canonical_book_ids = records['canonical_book_id'].nunique()

print(f"Number of unique bookIDs: {num_bookIDs}")
print(f"Number of unique canonical_book_ids: {num_canonical_book_ids}")


In [ ]:
records.nunique()

dataset reduction

In [ ]:
rec = records.copy()

In [ ]:

# Filter users with at least 5 ratings
user_rating_counts = records.groupby('userID').size().reset_index(name='rating_count')
active_users = user_rating_counts[user_rating_counts['rating_count'] >= 5]['userID']
df_active_users = records[records['userID'].isin(active_users)]

# Identify top N books
book_rating_counts = df_active_users.groupby('canonical_book_id').size().reset_index(name='rating_count')
top_books = book_rating_counts.sort_values(by='rating_count', ascending=False)

# Adjust N to approach 10,000 records
N = 500
top_N_books = top_books.head(N)['canonical_book_id']
df_top_books = df_active_users[df_active_users['canonical_book_id'].isin(top_N_books)]

# Check the number of records
total_records = df_top_books.shape[0]
print(f"Total records after filtering: {total_records}")




get a sample of 10000 records

In [ ]:
# Sample
if total_records > 10000:
    df_sampled = df_top_books.sample(n=10000, random_state=42)
else:
    df_sampled = df_top_books



use faker for assign synthetic data for timestamp for recommendation purposes

In [ ]:
from faker import Faker
from tqdm import tqdm
import datetime

# Initialize Faker and set seed for reproducibility
fake = Faker()
Faker.seed(42)

# Assuming 'df' is your DataFrame
num_rows = df_sampled.shape[0]

# Define date range
start_date = datetime.date(2014, 1, 1)
end_date = datetime.date(2019, 12, 31)

# Generate random dates using date_between
timestamps = [fake.date_between(start_date=start_date, end_date=end_date) for _ in tqdm(range(num_rows), desc="Generating timestamps")]

# Assign the timestamps to the DataFrame
df_sampled['timestamp'] = timestamps


In [ ]:
# Save the reduced dataset
df_sampled.to_csv('reduced_dataset.csv', index=False)

In [ ]:
df_sampled=pd.read_csv('/content/drive/MyDrive/IRWA/reduced_dataset.csv')

In [ ]:


#Calculate number of ratings per book
book_rating_counts = df_sampled.groupby('canonical_book_id').size().reset_index(name='rating_count')

#Merge with the original dataset
df = pd.merge(df_sampled, book_rating_counts, on='canonical_book_id', how='left')

#Calculate number of ratings per user
user_rating_counts = df_sampled.groupby('userID').size().reset_index(name='rated_books_count')

#Merge with the original dataset
df = pd.merge(df, user_rating_counts, on='userID', how='left')


In [ ]:
df.head(2)

# EDA

In [ ]:
# basic statics
print(df['rating'].describe())

In [ ]:
num_users = df['userID'].nunique()
num_books = df['canonical_book_id'].nunique()
print(f"Number of unique users: {num_users}")
print(f"Number of unique books: {num_books}")


In [ ]:
import matplotlib.pyplot as plt

df['rating'].hist(bins=5, edgecolor='black')
plt.title('Distribution of Ratings')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.show()


In [ ]:
#Top 10 books with most ratings
top_books = df.groupby('canonical_book_id').size().reset_index(name='rating_count')
top_books = top_books.sort_values(by='rating_count', ascending=False).head(8)
top_books = pd.merge(top_books, df[['canonical_book_id', 'bookTitle']].drop_duplicates(), on='canonical_book_id', how='left')
print(top_books[['bookTitle', 'rating_count']])


In [ ]:
top_books.plot(kind='barh', x='bookTitle', y='rating_count', legend=False)
plt.title('Top 10 Most Rated Books')
plt.xlabel('Number of Ratings')
plt.ylabel('Book Title')
plt.gca().invert_yaxis()
plt.show()


In [ ]:
#top 10 most engaged users
top_users = df.groupby('userID').size().reset_index(name='rating_count')
top_users = top_users.sort_values(by='rating_count', ascending=False).head(10)
print(top_users)

In [ ]:
top_users.plot(kind='barh', x='userID', y='rating_count', legend=False)
plt.title('Top 10 Most Active Users')
plt.xlabel('Number of Ratings Given')
plt.ylabel('User ID')
plt.gca().invert_yaxis()
plt.show()


In [ ]:
# rating frequency per user
user_counts = df.groupby('userID').size()
user_counts.hist(bins=50)
plt.title('Number of Ratings per User')
plt.xlabel('Number of Ratings')
plt.ylabel('Frequency')
plt.show()


In [ ]:
#ratings destribution per book
book_counts = df.groupby('canonical_book_id').size()
book_counts.hist(bins=50)
plt.title('Number of Ratings per Book')
plt.xlabel('Number of Ratings')
plt.ylabel('Frequency')
plt.show()

top rated books with low number of ratings

In [ ]:
#avarage rating per book
avg_book_ratings = df.groupby('canonical_book_id')['rating'].mean().reset_index(name='average_rating')


In [ ]:
# Merge with the number of ratings per book
avg_book_ratings = pd.merge(avg_book_ratings, book_rating_counts, on='canonical_book_id', how='left')

In [ ]:
# Filter books with at least 5 ratings
popular_books = avg_book_ratings[avg_book_ratings['rating_count'] >= 5]

In [ ]:
# Sort by average rating
top_rated_books = popular_books.sort_values(by='average_rating', ascending=False).head(10)

In [ ]:
# Get book titles
top_rated_books = pd.merge(top_rated_books, df[['canonical_book_id', 'bookTitle']].drop_duplicates(), on='canonical_book_id', how='left')

In [ ]:
print(top_rated_books[['bookTitle', 'average_rating', 'rating_count']])

correlation between Number of ratings and avarage ratings

In [ ]:
plt.scatter(popular_books['rating_count'], popular_books['average_rating'], alpha=0.5)
plt.title('Average Rating vs. Number of Ratings')
plt.xlabel('Number of Ratings')
plt.ylabel('Average Rating')
plt.show()


timebase analysis

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year'] = df['timestamp'].dt.year

In [ ]:
ratings_per_year = df.groupby('year').size().reset_index(name='num_ratings')
plt.plot(ratings_per_year['year'], ratings_per_year['num_ratings'])
plt.title('Number of Ratings Over Time')
plt.xlabel('Year')
plt.ylabel('Number of Ratings')
plt.show()

Heatmap of user-book interaction

In [ ]:
 user_book_matrix = df.pivot_table(index='userID', columns='bookID', values='rating')

In [ ]:
import seaborn as sns

# Select a subset for visualization (e.g., first 50 users and books)
subset = user_book_matrix.iloc[:50, :50]
sns.heatmap(subset, cmap='YlGnBu')
plt.title('Heatmap of User-Book Ratings (First 50 users and books)')
plt.xlabel('Book ID')
plt.ylabel('User ID')
plt.show()


Genre analysis

In [ ]:

genre_counts = df['bookCategory'].value_counts().reset_index()
genre_counts.columns = ['category', 'count']
genre_counts.head(10).plot(kind='barh', x='category', y='count', legend=False)
plt.title('Top 10 Categories')
plt.xlabel('Number of Ratings')
plt.ylabel('Category')
plt.gca().invert_yaxis()
plt.show()


word cloud of book titles

In [ ]:
from wordcloud import WordCloud

text = ' '.join(df['bookTitle'].dropna().tolist())
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

plt.figure(figsize=(15, 7.5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Book Titles')
plt.show()


user-rating patters

In [ ]:
avg_user_ratings = df.groupby('userID')['rating'].mean().reset_index(name='average_rating')
avg_user_ratings['average_rating'].hist(bins=50)
plt.title('Distribution of Average User Ratings')
plt.xlabel('Average Rating')
plt.ylabel('Number of Users')
plt.show()


#Tags building

In [ ]:
df.head(2)

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import re

nlp = spacy.load('en_core_web_sm')

# Function to expand common abbreviations in titles, publishers, etc.
def expand_abbreviation(text):
    abbreviations = {

        "intl": "international",
        "inc": "incorporated",
        "ltd": "limited",
        "co": "company",
        "sci-fi": "science fiction",

    }
    for abbr, expanded in abbreviations.items():
        text = re.sub(r'\b' + re.escape(abbr) + r'\b', expanded, text)
    return text

# Define separate cleaning functions for each column type
def clean_description(text):
    if not isinstance(text, str):
        return ''
    # Lowercase conversion
    text = text.lower()
    # Tokenize and remove stop words, punctuation, special characters
    doc = nlp(text)
    tokens = [
        token.lemma_ for token in doc
        if not token.is_stop and not token.is_punct and not token.like_num and len(token) > 2
    ]
    cleaned_text = ' '.join(tokens)
    return cleaned_text

def clean_publisher(text):
    if not isinstance(text, str):
        return ''
    # Lowercase and expand abbreviations
    text = text.lower()
    text = expand_abbreviation(text)
    # Tokenize and remove stop words, punctuation, special characters
    doc = nlp(text)
    tokens = [
        token.lemma_ for token in doc
        if not token.is_punct and not token.like_num
    ]
    cleaned_text = ' '.join(tokens)
    return cleaned_text

def clean_categories(text):
    if not isinstance(text, str):
        return ''
    # Split categories based on delimiter and clean whitespace
    tokens = text.split(",")  # Assume categories are comma-separated
    tokens = [token.strip().lower() for token in tokens if token.strip()]
    # Optionally, you can further process tokens (e.g., lemmatization)
    cleaned_text = ' '.join(tokens)
    return cleaned_text

def clean_authors(text):
    if not isinstance(text, str):
        return ''
    # Lowercase and expand abbreviations if needed
    text = text.lower()
    text = expand_abbreviation(text)
    # Tokenize and remove stop words, punctuation, special characters
    doc = nlp(text)
    tokens = [
        token.lemma_ for token in doc
        if not token.is_punct and not token.like_num
    ]
    cleaned_text = ' '.join(tokens)
    return cleaned_text

# Apply column-specific cleaning functions
df['description'] = df['description'].apply(clean_description)
df['publisher'] = df['publisher'].apply(clean_publisher)
df['categories'] = df['categories'].apply(clean_categories)
df['authors'] = df['authors'].apply(clean_authors)

# Example of how the dataset could be cleaned for tokenization and text normalization
df.head(2)  # This will show the cleaned dataset for review


In [ ]:

columns_to_combine = ['description', 'Title', 'authors', 'publisher', 'categories']

# Combine the columns into a new 'combined_text' column
df['tags'] = df[columns_to_combine].agg(' '.join, axis=1)

# Remove extra whitespace
df['tags'] = df['tags'].str.replace('\s+', ' ', regex=True).str.strip()

# Display the DataFrame to verify
print(df[['tags']].head())


# Rating Based Recommendations

In [148]:
df = pd.read_csv('newwww.csv')

In [150]:
df.columns

Index(['userID', 'canonical_book_id', 'userName', 'bookID', 'rating',
       'bookTitle', 'bookAuthor', 'bookCategory', 'bookPublisher',
       'description', 'image', 'Title', 'authors', 'publishedDate', 'Price',
       'publisher', 'categories', 'timestamp', 'rating_count',
       'rated_books_count', 'tags', 'average_ratings', 'weighted_rating'],
      dtype='object')

**Weighted rating formula for most popular books with highest rating**

`Weighted Rating (WR) = (v/(v+m)) * R + (m/(v+m)) * C`

Where:

*   R = average rating for the book
*   v = number of ratings for the book
*   m = minimum number of ratings required to be
     considered
*   C = the mean rating across all books

In [153]:
# Calculate the average rating for each book
#avrg_ratings = df.groupby('bookID')['rating'].mean()

# Merge the results back to the original dataframe
#df = df.merge(avrg_ratings.rename('average_ratings'), on='bookID', how='left')

In [155]:
# Calculate the overall average rating for all books (C)
C = df['average_ratings'].mean()
print(C)

4.3131


In [157]:


# Set the minimum number of ratings required to be considered (m)
m = df['rating_count'].quantile(0.90)  # For example, taking the top 10% as the threshold


# Apply the weighted rating formula to get the weighted score (WR)
#df['weighted_rating'] = (df['rating_count'] / (df['rating_count'] + m)) * df['average_ratings'] + (m / (df['rating_count'] + m)) * C

# Sort the books by weighted rating and drop duplicates
rating_based_recommendations = df.sort_values(by='weighted_rating', ascending=False).drop_duplicates(subset='bookID').head(10)
# Select and display the relevant columns
final_columns = ['bookTitle','bookAuthor','bookCategory', 'Price', 'weighted_rating', 'image' ]
rating_based_recommendations= rating_based_recommendations[final_columns]


**Breakdown of the Code:**
*   rating_count: Counts how many ratings each book received.
*   average_ratings: Calculates the average rating of each book.
*   C: The global average rating for all books.
*   m: The minimum number of ratings needed to give more confidence to a rating (set at the 90th percentile of the rating counts).
* weighted_rating: Uses the weighted formula to give a better ranking to books that have more ratings and a good rating score.  
* rating_based_recommendations: The books are sorted based on the weighted rating, and then the top 10 are selected.

In [160]:
print("Rating Base Recommendation System: ")
rating_based_recommendations.head(10)

Rating Base Recommendation System: 


,bookTitle,bookAuthor,bookCategory,Price,weighted_rating,image
7552,Night,Clement Moore,['Juvenile Fiction'],40.68,4.600841,http://books.google.com/books/content?id=LSSyD...
3325,George Orwell 1984,George Orwell,['Fiction'],5.13,4.593366,http://books.google.com/books/content?id=PEpoE...
2456,Nineteen Eighty-Four,George Orwell,['Totalitarianism'],24.49,4.582728,http://books.google.com/books/content?id=h7BUN...
544,"Moby-Dick: Or, the Whale",Herman Melville,['Fiction'],10.88,4.574315,http://books.google.com/books/content?id=xJFPE...
1209,1984,George Orwell,['Fiction'],16.74,4.554396,http://books.google.com/books/content?id=gTx1A...
1937,The Hobbit,J. R. R. Tolkien,['Juvenile Fiction'],34.80,4.551917,http://books.google.com/books/content?id=LLSpn...
7633,The Lord of the Rings - Boxed Set,j.r.r. Tolkien,['Young Adult Fiction'],6.87,4.544882,http://books.google.com/books/content?id=CixXE...
322,John Adams,David McCullough,"['Biography', 'Autobiography']",29.94,4.522205,http://books.google.com/books/content?id=GHMnz...
6252,Harry Potter and the Sorcerer's Stone,J. K. Rowling,['Juvenile Fiction'],30.54,4.517416,http://books.google.com/books/content?id=HksgD...
474,Harry Potter and the Chamber of Secrets,J. K. Rowling,['Juvenile Fiction'],22.73,4.514026,http://books.google.com/books/content?id=nmXTy...


In [162]:
highest_rated = rating_based_recommendations.to_csv("highest_rated.csv", index=False)

# Content Based Recommendations

In [165]:
df.columns

Index(['userID', 'canonical_book_id', 'userName', 'bookID', 'rating',
       'bookTitle', 'bookAuthor', 'bookCategory', 'bookPublisher',
       'description', 'image', 'Title', 'authors', 'publishedDate', 'Price',
       'publisher', 'categories', 'timestamp', 'rating_count',
       'rated_books_count', 'tags', 'average_ratings', 'weighted_rating'],
      dtype='object')

In [167]:
dfc=df.copy()

In [169]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\D E L
[nltk_data]     L\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [170]:
# Compute TF-IDF and cosine similarity
def compute_similarity(text_data):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix_content = tfidf_vectorizer.fit_transform(text_data)
    cosine_similarities_content = cosine_similarity(tfidf_matrix_content, tfidf_matrix_content)
    return cosine_similarities_content

In [171]:
cosine_similarities_content = compute_similarity(df['tags'])

In [172]:
cosine_similarities_content

array([[1.        , 0.0136235 , 0.01225344, ..., 0.05493275, 0.03175353,
        0.01076612],
       [0.0136235 , 1.        , 0.01313919, ..., 0.        , 0.04691696,
        0.01467168],
       [0.01225344, 0.01313919, 1.        , ..., 0.01717518, 0.0294293 ,
        0.03010292],
       ...,
       [0.05493275, 0.        , 0.01717518, ..., 1.        , 0.01795643,
        0.04391316],
       [0.03175353, 0.04691696, 0.0294293 , ..., 0.01795643, 1.        ,
        0.01999064],
       [0.01076612, 0.01467168, 0.03010292, ..., 0.04391316, 0.01999064,
        1.        ]])

In [174]:
stop_words = set(stopwords.words('english'))
# Text preprocessing function
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

In [202]:
def contentBased_recommendations(title, cosine_similarities, df, top_n=10):
    # Preprocess the input title
    input_title = preprocess_text(title)


    if not input_title:
        print(f"Title '{title}' not found in the dataset.")
        return pd.DataFrame()

    # Create a temporary column for processed titles
    df['processed_title'] = df['bookTitle'].apply(preprocess_text)

    # Find the index of the input title
    idx_list = df[df['processed_title'] == input_title].index.tolist()
    if not idx_list:
        print(f"Title '{title}' not found in the dataset.")
        df.drop('processed_title', axis=1, inplace=True)
        return pd.DataFrame()

    # use the first occurrence
    idx = idx_list[0]

    # Get similarity scores for all books
    sim_scores = list(enumerate(cosine_similarities[idx]))

    # Exclude the book itself and any other books with the same normalized title
    sim_scores = [score for score in sim_scores if score[0] not in idx_list]

    # Sort the books based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

   # Keep track of unique book titles
    unique_titles = set()
    filtered_sim_scores = []
    for i, score in sim_scores:
        book_title = df.iloc[i]['Title']
        if book_title not in unique_titles:
            unique_titles.add(book_title)
            filtered_sim_scores.append((i, score))
        # Stop if we have enough recommendations
        if len(filtered_sim_scores) >= top_n:
            break

    # Get the indices and scores of the top_n unique similar books
    book_indices = [i for i, _ in filtered_sim_scores]
    similarity_scores = [score for _, score in filtered_sim_scores]

    # Get the recommended books with their scores
    recommendations = df.iloc[book_indices].copy()
    recommendations['score_content'] = similarity_scores

    # Drop the temporary column
    df.drop('processed_title', axis=1, inplace=True)

    return recommendations

**GET CONTENT_BASED RECOMMENDATIONS FOR A SPECIFIC BOOK**

In [205]:


# Get recommendations as a DataFrame
title = 'The Hobbit'  # Replace with an actual title from your dataset
recommendations_df = contentBased_recommendations(title, cosine_similarities_content, df)

# Select desired columns
desired_columns = ['bookID','bookTitle','bookAuthor','bookCategory', 'Price', 'weighted_rating', 'image']
recommendations_df = recommendations_df[desired_columns]

# Check if recommendations are found
if recommendations_df.empty:
    print(f"No recommendations found for '{title}'.")
else:
    # Display the DataFrame
    print(f"Recommendations for '{title}':")
      # Display up to 10 recommendations
    print(recommendations_df.head(10).to_string())


Recommendations for 'The Hobbit':
         bookID                                                         bookTitle                 bookAuthor             bookCategory  Price  weighted_rating                                                                                                            image
509  B000GQK706                                 The Lord of the Rings - Boxed Set             j.r.r. Tolkien  ['Young Adult Fiction']   6.87         4.544882            http://books.google.com/books/content?id=CixXEAAAQBAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api
28   B000Q08CDQ  Return of the King Being the Third Part of the Lord of the Rings           J. R. R. Tolkien              ['Fiction']  38.77         4.415748            http://books.google.com/books/content?id=1EcBuwAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api
108  B000NWU3I4  The Hobbitt, or There and Back Again; Illustrated by the Author.           J. R. R. Tolkien              ['Fiction']  39.31         4.51

# Collaborative Filtering Recommendations

In [185]:
df.columns

Index(['userID', 'canonical_book_id', 'userName', 'bookID', 'rating',
       'bookTitle', 'bookAuthor', 'bookCategory', 'bookPublisher',
       'description', 'image', 'Title', 'authors', 'publishedDate', 'Price',
       'publisher', 'categories', 'timestamp', 'rating_count',
       'rated_books_count', 'tags', 'average_ratings', 'weighted_rating'],
      dtype='object')

In [187]:
from surprise import SVD, Dataset, Reader

def collaborative_filtering_recommendations(df, target_user_id, top_n=10):
    # Prepare the data
    reader = Reader(rating_scale=(df['rating'].min(), df['rating'].max()))
    data = Dataset.load_from_df(df[['userID', 'bookID', 'rating']], reader)
    trainset = data.build_full_trainset()

    # Build the model
    algo = SVD()
    algo.fit(trainset)

    # Get a list of all book IDs
    all_book_ids = df['bookID'].unique()

    # Predict ratings for all books not rated by the user
    user_rated_books = df[df['userID'] == target_user_id]['bookID'].unique()
    books_to_predict = [iid for iid in all_book_ids if iid not in user_rated_books]

    predictions = [algo.predict(target_user_id, iid) for iid in books_to_predict]

    # Get top N recommendations
    predictions.sort(key=lambda x: x.est, reverse=True)
    top_predictions = predictions[:top_n]
    top_book_ids = [pred.iid for pred in top_predictions]
    scores = [pred.est for pred in top_predictions]

    # Get book details
    recommendations = df[df['bookID'].isin(top_book_ids)].drop_duplicates('bookID')
    recommendations = recommendations[['bookID','bookTitle','bookAuthor','bookCategory', 'Price', 'weighted_rating', 'image']].copy()
    recommendations['score_collaborative'] = scores

    return recommendations


**COLLABORATIVE FILTERING TEST**

EVALUATION

In [191]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate, KFold
from collections import defaultdict

# Load data
reader = Reader(rating_scale=(df['rating'].min(), df['rating'].max()))
data = Dataset.load_from_df(df[['userID', 'bookID', 'rating']], reader)

# Define algorithm
algo = SVD()

# Step 3: Evaluate with RMSE and MAE using cross-validation
cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print("Average RMSE:", cv_results['test_rmse'].mean())
print("Average MAE:", cv_results['test_mae'].mean())

# Step 4: Compute Precision@K and Recall@K
def precision_recall_at_k(predictions, k=10, threshold=4.0):
    '''Return precision and recall at k metrics for each user.'''

    # Map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

kf = KFold(n_splits=5)

precisions_list = []
recalls_list = []

for trainset, testset in kf.split(data):
    # Train the algorithm
    algo.fit(trainset)

    # Test the algorithm
    predictions = algo.test(testset)

    # Compute precision and recall
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=4.0)

    # Average precision and recall for this fold
    precision = sum(prec for prec in precisions.values()) / len(precisions)
    recall = sum(rec for rec in recalls.values()) / len(recalls)

    precisions_list.append(precision)
    recalls_list.append(recall)

# Compute overall average precision and recall
avg_precision = sum(precisions_list) / len(precisions_list)
avg_recall = sum(recalls_list) / len(recalls_list)

print(f'Average Precision@10: {avg_precision:.4f}')
print(f'Average Recall@10: {avg_recall:.4f}')


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8684  0.8591  0.8545  0.8266  0.8739  0.8565  0.0164  
MAE (testset)     0.6568  0.6522  0.6504  0.6383  0.6573  0.6510  0.0069  
Fit time          0.12    0.08    0.07    0.07    0.08    0.08    0.02    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    
Average RMSE: 0.8565152638485873
Average MAE: 0.6510054033868172
Average Precision@10: 0.7893
Average Recall@10: 0.7895


TESTING

In [194]:
target_user_id = 'AZSV99SDJC242'
top_n = 10
recommendations = collaborative_filtering_recommendations(df, target_user_id, top_n=top_n)
print(f"Top {top_n} recommendations for user {target_user_id}:")
recommendations

Top 10 recommendations for user AZSV99SDJC242:


,bookID,bookTitle,bookAuthor,bookCategory,Price,weighted_rating,image,score_collaborative
13,721417299,Christmas Carol,Charles Dickens,['Childrens Stories'],13.06,4.489513,http://books.google.com/books/content?id=Yg_4w...,4.676813
79,808598104,The Killer Angels,Michael Shaara,['Fiction'],13.80,4.490088,http://books.google.com/books/content?id=wNE8P...,4.639634
88,B000TNGU5M,Night,Clement Moore,['Juvenile Fiction'],40.68,4.600841,http://books.google.com/books/content?id=LSSyD...,4.630193
133,B000N79NHQ,Animal Farm 50th Anniversary Edition,George Orwell,['Fiction'],24.19,4.508823,http://books.google.com/books/content?id=Ka5iF...,4.612696
246,B0000632ZJ,Lonesome Dove,Larry McMurtry,['Fiction'],36.93,4.464135,http://books.google.com/books/content?id=rZohf...,4.593668
321,446781819,Gone With the Wind,Margaret Mitchell,['Fiction'],34.16,4.489513,http://books.google.com/books/content?id=01KdD...,4.588171
330,B000HKRIJO,Mere Christianity,C. S. Lewis,['Religion'],16.51,4.468234,http://books.google.com/books/content?id=OF-YS...,4.584618
433,B0007BY9UY,Homage to Catalonia,George Orwell,"['Biography', 'Autobiography']",30.72,4.442714,http://books.google.com/books/content?id=Ybr7D...,4.570064
509,B000GQK706,The Lord of the Rings - Boxed Set,j.r.r. Tolkien,['Young Adult Fiction'],6.87,4.544882,http://books.google.com/books/content?id=CixXE...,4.567467
1042,B000NOXDUC,The Rise and Fall of the Third Reich - 2 Volum...,William L. Shirer,['History'],43.92,4.469761,http://books.google.com/books/content?id=kWhsD...,4.553456


# Hybird Recommendations

In [197]:
df.columns

Index(['userID', 'canonical_book_id', 'userName', 'bookID', 'rating',
       'bookTitle', 'bookAuthor', 'bookCategory', 'bookPublisher',
       'description', 'image', 'Title', 'authors', 'publishedDate', 'Price',
       'publisher', 'categories', 'timestamp', 'rating_count',
       'rated_books_count', 'tags', 'average_ratings', 'weighted_rating'],
      dtype='object')

In [225]:
from sklearn.preprocessing import MinMaxScaler

#  Generate Content-Based Recommendations with Scores
book_title = 'The Hobbit'
content_recommendations = contentBased_recommendations(title, cosine_similarities_content, df, top_n=50)

# Generate Collaborative Filtering Recommendations with Scores
target_user_id = 'AHD101501WCN1'
collaborative_recommendations = collaborative_filtering_recommendations(df, target_user_id, top_n=50)

# Normalize, Merge, and Combine Recommendations

# Normalize scores
def normalize_scores(df, score_column):
    scaler = MinMaxScaler()
    df[score_column] = scaler.fit_transform(df[[score_column]])
    return df

content_recommendations = normalize_scores(content_recommendations, 'score_content')
collaborative_recommendations = normalize_scores(collaborative_recommendations, 'score_collaborative')

# Ensure 'bookID's are strings
content_recommendations['bookID'] = content_recommendations['bookID'].astype(str)
collaborative_recommendations['bookID'] = collaborative_recommendations['bookID'].astype(str)

# Check for missing 'bookID's
if content_recommendations['bookID'].isnull().any():
    print("Null 'bookID's found in content_recommendations")
    # Handle accordingly, e.g., drop rows with null 'bookID's
    content_recommendations = content_recommendations.dropna(subset=['bookID'])

if collaborative_recommendations['bookID'].isnull().any():
    print("Null 'bookID's found in collaborative_recommendations")
    collaborative_recommendations = collaborative_recommendations.dropna(subset=['bookID'])

# Verify 'score_content' and 'score_collaborative' columns exist
if 'score_content' not in content_recommendations.columns:
    print("'score_content' column is missing in content_recommendations")

if 'score_collaborative' not in collaborative_recommendations.columns:
    print("'score_collaborative' column is missing in collaborative_recommendations")

# Merge recommendations
combined_recommendations = pd.merge(
    content_recommendations,
    collaborative_recommendations,
    on='bookID',
    how='outer',
    suffixes=('_content', '_collaborative')
)

# Fill missing scores
combined_recommendations['score_content'] = combined_recommendations['score_content'].fillna(0)
combined_recommendations['score_collaborative'] = combined_recommendations['score_collaborative'].fillna(0)

# Calculate weighted scores
content_weight = 0.5
collaborative_weight = 0.5

combined_recommendations['hybrid_score'] = (
    content_weight * combined_recommendations['score_content'] +
    collaborative_weight * combined_recommendations['score_collaborative']
)

# Sort and select top recommendations
combined_recommendations = combined_recommendations.sort_values(by='hybrid_score', ascending=False)

# Filter out books without titles
#combined_recommendations = combined_recommendations[combined_recommendations['bookTitle'].notna()]

# Reset index after filtering
combined_recommendations.reset_index(drop=True, inplace=True)

# Ensure we have enough recommendations
top_n = 10
if len(combined_recommendations) < top_n:
    top_n = len(combined_recommendations)
top_recommendations = combined_recommendations.head(top_n)



In [229]:
top_recommendations

,userID,canonical_book_id,userName,bookID,rating,bookTitle_content,bookAuthor_content,bookCategory_content,bookPublisher,description,...,processed_title,score_content,bookTitle_collaborative,bookAuthor_collaborative,bookCategory_collaborative,Price_collaborative,weighted_rating_collaborative,image_collaborative,score_collaborative,hybrid_score
0,A2P6KAWXJ16234,2474.0,Kelly (Fantasy Literature),B000GQK706,4.0,The Lord of the Rings - Boxed Set,j.r.r. Tolkien,['Young Adult Fiction'],HarperCollins,beautiful gift edition hobbit j.r.r tolkien cl...,...,lord rings boxed set,1.000000,The Lord of the Rings - Boxed Set,j.r.r. Tolkien,['Young Adult Fiction'],6.87,4.544882,http://books.google.com/books/content?id=CixXE...,0.520158,0.760079
1,A2TAPSNKK9AFSQ,2970.0,"John Baranyai ""Book Nerd And Ex High School G...",B000I1V604,5.0,One Flew Over the Cuckoo's Nest,Ken Kesey,['Fiction'],Penguin,inspiration new netflix original series ratche...,...,one flew cuckoos nest,0.475820,One Flew Over the Cuckoo's Nest,Ken Kesey,['Fiction'],15.80,4.399619,http://books.google.com/books/content?id=KhRvD...,0.765484,0.620652
2,A1POFVVXUZR3IQ,1824.0,Z Hayes,B000IEZE3G,5.0,Harry Potter and the Sorcerer's Stone,J. K. Rowling,['Juvenile Fiction'],Bloomsbury Publishing,celebrate year harry potter magic harry potter...,...,harry potter sorcerers stone,0.136002,Harry Potter and the Sorcerer's Stone,J. K. Rowling,['Juvenile Fiction'],30.54,4.517416,http://books.google.com/books/content?id=HksgD...,1.000000,0.568001
3,A3P9D22KDEXVMN,718.0,"Happy Chappy ""An Avid Reader.""",721417299,5.0,Christmas Carol,Charles Dickens,['Childrens Stories'],Ladybird,ladybird classic abridged retelling classic st...,...,christmas carol,0.076380,Christmas Carol,Charles Dickens,['Childrens Stories'],13.06,4.489513,http://books.google.com/books/content?id=Yg_4w...,1.000000,0.538190
4,NaN,NaN,NaN,B000NHNM3C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,George Orwell 1984,George Orwell,['Fiction'],5.13,4.593366,http://books.google.com/books/content?id=PEpoE...,1.000000,0.500000
5,NaN,NaN,NaN,B0000ANLB7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,To the Nines,Janet Evanovich,['Fiction'],14.62,4.389777,http://books.google.com/books/content?id=jKx9f...,1.000000,0.500000
6,NaN,NaN,NaN,B00005UVH9,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,A Fine Balance,Rohinton Mistry,['Fiction'],38.66,4.429607,http://books.google.com/books/content?id=WcaFd...,1.000000,0.500000
7,NaN,NaN,NaN,B000NDSX6C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,The Hobbit,J. R. R. Tolkien,['Juvenile Fiction'],34.80,4.551917,http://books.google.com/books/content?id=LLSpn...,1.000000,0.500000
8,NaN,NaN,NaN,B0000632ZJ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,Lonesome Dove,Larry McMurtry,['Fiction'],36.93,4.464135,http://books.google.com/books/content?id=rZohf...,1.000000,0.500000
9,NaN,NaN,NaN,B000MZWXNA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,The Martian Way,Isaac Asimov,['Fiction'],38.20,4.388064,http://books.google.com/books/content?id=oEMGH...,1.000000,0.500000
